## Data Cleaning in Python (Pandas)

#### Indroduction
This project focuses on cleaning a dataset that contains information about the top Netflix movies and TV shows as listend on IMDb. The dataset was successfully scraped from the IMDb website. It includes 9 columns that describe various attributes of the movies and TV shows.

The dataset includes the following columns:
- 'MOVIES': The title
- 'YEAR': The release year
- 'GENRE': The primary genre
- 'RATING': The IBDb rating
- 'ONE-LINE': A one-line description
- 'STARS': The director and stars
- 'VOTES': The number of votes recieved
- 'RunTime': The runtime
- 'Gross': The total amount grossed worldwide


#### Cleaning objectives
The primary objective of this project is to clean the dataset to ensure its suitability for further analysis and modeling. The cleaning process involves the following steps:
1. **Fixing Data Types**
2. **Splitting Columns**
3. **Dropping Certain Rows with Null Values**
4. **Reordering Columns**
5. **Resetting Indices**

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"/Users/aidanturner/Desktop/Data Analytics Projects/data_cleaning_practice/movies.csv")

## Dataset Prior to Cleaning:

In [3]:
df

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9994,The Imperfects,(2021– ),"\nAdventure, Drama, Fantasy",NaN,\nAdd a Plot\n,\n \n Stars:\nMorgan Taylor Camp...,NaN,NaN,NaN
9995,Arcane,(2021– ),"\nAnimation, Action, Adventure",NaN,\nAdd a Plot\n,\n,NaN,NaN,NaN
9996,Heart of Invictus,(2022– ),"\nDocumentary, Sport",NaN,\nAdd a Plot\n,\n Director:\nOrlando von Einsiedel\n| \n ...,NaN,NaN,NaN
9997,The Imperfects,(2021– ),"\nAdventure, Drama, Fantasy",NaN,\nAdd a Plot\n,\n Director:\nJovanka Vuckovic\n| \n Sta...,NaN,NaN,NaN


In [4]:
df = df.drop_duplicates()

### Clean the 'YEAR' column

In [5]:
import re

# Custom function to process each entry
def normalize_year(year_str):
    if not isinstance(year_str, str):
        return year_str    # Return as is if it's not a string

    # Remove parentheses if present
    year_str = year_str.strip('()')

    # Use regex to match different patterns
    match = re.match(r'(\d{4})\s*[--]?\s*(\d{4})?', year_str)
    if match:
        start_year, end_year = match.groups()
        return end_year if end_year else start_year
    
    return year_str.strip()   # Handle single years or any other format

# Convert all values to strings and then apply to function
df["YEAR"] = df["YEAR"].astype(str).apply(normalize_year)

/var/folders/yb/dh4rkk5x2pl9wqk2khnmv0sh0000gn/T/ipykernel_11974/732165580.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["YEAR"] = df["YEAR"].astype(str).apply(normalize_year)


### Clean the 'GENRE' column

In [6]:
# Remove leading newlines and split the 'GENRE' column into multiple columns
genres_split = df['GENRE'].str.strip().str.split(', ', expand=True)

# Rename the resulting columns
genres_split.columns = ['GENRE_1', 'GENRE_2', 'GENRE_3']

# Join the new columns back to the original dataframe
df = df.join(genres_split)

# Drop the originial 'GENRE' column
df.drop(columns=['GENRE'], inplace=True)

### Clean the 'ONE-LINE' column

In [7]:
import numpy as np

# Remove leading newlines
df['ONE-LINE'] = df['ONE-LINE'].str.strip()

# Replace 'Add a Plot' with NaN
df['ONE-LINE'] = df['ONE-LINE'].replace('Add a Plot', np.nan)

### Clean the 'STARS' column

In [8]:
import re

# Function to extract director and stars
def extract_director_stars(entry):
    if not isinstance(entry, str):
        return None, [None, None]

    # Use regex to find the director name/star name in the entry
    director_match = re.search(r'Director:\n([^\n|]+)', entry)
    stars_match = re.search(r'Stars:\n(.+)', entry, re.DOTALL)

    # Extract the director name if found
    director = director_match.group(1).strip() if director_match else None

    if stars_match:
        stars_raw = stars_match.group(1).strip().split(', \n')
        stars = [star.strip() for star in stars_raw if star.strip()]
    else:
        stars = []

    while len(stars) < 2:
        stars.append(None)

    return director, stars[:2]

# Apply the function to the 'STARS' column and expand the results into separate columns
df[['DIRECTOR', 'STARS']] = df['STARS'].apply(lambda x: pd.Series(extract_director_stars(x)))

# Split the 'STARS' column into individual star columns
stars_df = pd.DataFrame(df['STARS'].tolist(), columns=['STAR_1', 'STAR_2'])

# Combine the new columns back into the original dataframe and drop the 'STARS' column
df = pd.concat([df.drop(columns=['STARS']), stars_df], axis=1)

### Clean the 'VOTES' column

In [9]:
# Remove commas
df['VOTES'] = df['VOTES'].str.replace(',', '')

# Convert to numeric, keeping NaN values
df['VOTES'] = pd.to_numeric(df['VOTES'], errors='coerce').astype('Int64')

### Clean rows with null values and reorder the columns

In [10]:
# Drop rows with null values in the specified columns
df = df.dropna(subset=['MOVIES', 'YEAR', 'RATING', 'VOTES', 'GENRE_1'])

# Reset the indices
df.reset_index(drop=True, inplace=True)

# Drop the 'Gross' column
df = df.drop(columns=['Gross'])

# Reorder the columns
new_column_order = ['MOVIES', 'YEAR', 'GENRE_1', 'GENRE_2', 'GENRE_3', 'RATING', 'ONE-LINE', 'DIRECTOR', 'STAR_1', 'STAR_2', 'VOTES', 'RunTime']
df = df[new_column_order]

## Dataset After Cleaning:

In [11]:
df

,MOVIES,YEAR,GENRE_1,GENRE_2,GENRE_3,RATING,ONE-LINE,DIRECTOR,STAR_1,STAR_2,VOTES,RunTime
0,Blood Red Sky,2021,Action,Horror,Thriller,6.1,A woman with a mysterious illness is forced in...,Peter Thorwarth,Peri Baumeister,Carl Anton Koch,21062,121.0
1,Masters of the Universe: Revelation,2021,Animation,Action,Adventure,5.0,The war for Eternia begins again in what may b...,None,Chris Wood,Sarah Michelle Gellar,17870,25.0
2,The Walking Dead,2010,Drama,Horror,Thriller,8.2,Sheriff Deputy Rick Grimes wakes up from a com...,None,Andrew Lincoln,Norman Reedus,885805,44.0
3,Rick and Morty,2013,Animation,Adventure,Comedy,9.2,An animated series that follows the exploits o...,None,Justin Roiland,Chris Parnell,414849,23.0
4,Outer Banks,2020,Action,Crime,Drama,7.6,A group of teenagers from the wrong side of th...,None,Chase Stokes,Madelyn Cline,25858,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8152,Sexify,2021,Comedy,Drama,None,7.1,After facing their parents at Easter breakfast...,Kalina Alabrudzinska,NaN,NaN,103,37.0
8153,Sexify,2021,Comedy,Drama,None,6.9,After getting an anonymous tip about the copul...,Kalina Alabrudzinska,NaN,NaN,96,40.0
8154,Sexify,2021,Comedy,Drama,None,7.1,The big pitch at the university tech competiti...,Kalina Alabrudzinska,NaN,NaN,100,47.0
8155,Castlevania,2017,Animation,Action,Adventure,8.2,Lenore urges Hector to not test Camilla's pati...,Sam Deats,NaN,NaN,911,24.0


In [13]:
df.to_csv('cleaned_data.csv', index=False)